<a href="https://colab.research.google.com/github/Samin-Sadaf7/MasteringObjectDetection/blob/main/MateringObhectDetection_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bird Mulitple object Detection using FasterRCNN and Tensorflow Object Detection API

Part -2 : training

### Download model from tensorflow 2 model garden

In [1]:
import os
import re #regular expression
import tensorflow as tf
from google.colab import drive
#Mounting to Google Drive
drive.mount('/content/gdrive', force_remount=True)
#Set current directory to MyDrive , put the downloaded files there
%cd gdrive/MyDrive/
# Get current working dir
current_dir= os.getcwd()
print("Root dir:", current_dir)

Mounted at /content/gdrive
/content/gdrive/MyDrive
Root dir: /content/gdrive/MyDrive


In [2]:
#Define some constants
RANDOM_SEED = 99
BATCH_SIZE = 1
NUM_STEPS = 2100
NUM_EVAL_STEPS = 1000

In [5]:
# Check if the directory exists
if not os.path.isdir('faster_rcnn_resnet101_v1-640x640-coco17_tpu-8'):
    # Download the model
    !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz
    # Extract the compressed file
    !tar -xf faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz
    # Remove the compressed file
    !rm faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz

--2024-04-07 10:10:13--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.26.207, 172.217.203.207, 142.250.97.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.26.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353643040 (337M) [application/x-tar]
Saving to: ‘faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz.1’

faster_rcnn_resnet1 100%[===================>] 337.26M  51.7MB/s    in 6.1s    

2024-04-07 10:10:19 (55.2 MB/s) - ‘faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz.1’ saved [353643040/353643040]

